In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

# Load the data from a CSV file
data = pd.read_csv('diabetes_prediction_dataset.csv')

# Drop columns with any null or NaN values
data.dropna(axis=1, inplace=True)

# Encode categorical variables: gender and smoking_history
data['gender'] = data['gender'].map({'Female': 0, 'Male': 1})  # Map Female to 0 and Male to 1

# Map the smoking_history variable to numerical values
smoking_history_mapping = {
    'current': 1,
    'ever': 2,
    'former': 3,
    'never': 4,
    'No info': 5,
    'not current': 6
}

data['smoking_history'] = data['smoking_history'].map(smoking_history_mapping)

# Extract features (X) and target variable (y)
X = data.drop(columns=['diabetes']).values  # Drop the target column to get features
y = data['diabetes'].values                  # Target variable

# Check the unique counts of the target variable
unique_counts = np.unique(y, return_counts=True)
print(unique_counts)


(array([0, 1]), array([91500,  8500]))


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


Logestic Regression

Random Forest Classifier

In [4]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     27450
           1       1.00      0.68      0.81      2550

    accuracy                           0.97     30000
   macro avg       0.99      0.84      0.90     30000
weighted avg       0.97      0.97      0.97     30000



Train XgBOOST

In [5]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     27450
           1       0.95      0.70      0.81      2550

    accuracy                           0.97     30000
   macro avg       0.96      0.85      0.90     30000
weighted avg       0.97      0.97      0.97     30000



In [6]:
models = [
   
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [ ]:

mlflow.set_experiment("diabetes Detection 1")
mlflow.set_tracking_uri("https://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/10/24 16:25:20 INFO mlflow.tracking.fluent: Experiment with name 'diabetes Detection 1' does not exist. Creating a new experiment.


In [ ]:
model_name = 'XGB-Smote 1'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'
with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

In [ ]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

In [ ]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
a=y_pred[:4]
print(a)